In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycountry
from countryinfo import CountryInfo as CInfo 
import matplotlib.pyplot as plt
import  altair  as  alt 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
all_languages_panel = pd.read_csv("../output/csv/all_languages_panel.csv", encoding="iso-8859-1")
all_languages_panel = all_languages_panel.drop('Unnamed: 0', axis=1)
all_languages_panel.head()

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int,gpt_available,population,num_pushers_pc,post_CI,python,Treatment,post_1,post_2,gpt_available_post1,gpt_available_post2,Category
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1,1,1,9446000,21.871692,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2,2,1,9446000,34.448444,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3,3,1,9446000,33.961465,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4,4,1,9446000,34.459030,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1,5,1,9446000,35.157739,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)


In [35]:
# Crear una lista con los valores a mantener
valores_a_mantener = ['C', 'C#', 'C++','Go','Java','JavaScript','PHP','Python','Ruby','TypeScript']

# Filtrar el DataFrame para quedarse solo con las observaciones deseadas
df_filtrado = all_languages_panel[all_languages_panel['language'].isin(valores_a_mantener)]

In [36]:
df_filtrado = df_filtrado.rename(columns={'unique_id': 'unique_id_old'})
df_filtrado

,unique_id_old,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int,gpt_available,population,num_pushers_pc,post_CI,python,Treatment,post_1,post_2,gpt_available_post1,gpt_available_post2,Category
64,AE-JavaScript,2020-Q1,1734.0,AE,JavaScript,2020,1,1,1,9446000,18.356976,0,0,0,0,0,0,0,"High-Level, General-Purpose Languages"
65,AE-JavaScript,2020-Q2,2299.0,AE,JavaScript,2020,2,2,1,9446000,24.338344,0,0,0,0,0,0,0,"High-Level, General-Purpose Languages"
66,AE-JavaScript,2020-Q3,2366.0,AE,JavaScript,2020,3,3,1,9446000,25.047639,0,0,0,0,0,0,0,"High-Level, General-Purpose Languages"
67,AE-JavaScript,2020-Q4,2468.0,AE,JavaScript,2020,4,4,1,9446000,26.127461,0,0,0,0,0,0,0,"High-Level, General-Purpose Languages"
68,AE-JavaScript,2021-Q1,2544.0,AE,JavaScript,2021,1,5,1,9446000,26.932035,0,0,0,0,0,0,0,"High-Level, General-Purpose Languages"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4,12,0,25956000,0.000000,0,0,0,1,0,0,0,"High-Level, General-Purpose Languages"
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1,13,0,25956000,0.000000,0,0,0,1,1,0,0,"High-Level, General-Purpose Languages"
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2,14,0,25956000,0.000000,1,0,0,1,1,0,0,"High-Level, General-Purpose Languages"
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3,15,0,25956000,0.000000,1,0,0,1,1,0,0,"High-Level, General-Purpose Languages"


In [37]:
# Create a unique identifier
df_filtrado['unique_id'] = df_filtrado['iso2_code'] + '-' + df_filtrado['year_quarter']

# Get unique combined IDs
unique_ids = df_filtrado['unique_id'].unique()

# Group by the new combined ID and sum the 'num_pushers'
grouped_df = df_filtrado.groupby('unique_id').agg({'num_pushers': 'sum'}).reset_index()

# Extract iso2_code from combined_id
grouped_df[['iso2_code','year', 'quarter']] = grouped_df['unique_id'].str.split('-', expand=True)

# Keep the 'population' column for each 'iso2_code'
population_data = df_filtrado[['iso2_code', 'population']].drop_duplicates()

# Keep the 'population' column for each 'iso2_code'
gtp = df_filtrado[['iso2_code', 'gpt_available']].drop_duplicates()

# Merge to bring population into grouped_df
grouped_df = grouped_df.merge(population_data, on='iso2_code', how='left')

# Merge to bring population into grouped_df
grouped_df = grouped_df.merge(gtp, on='iso2_code', how='left')

# Display the result
grouped_df

,unique_id,num_pushers,iso2_code,year,quarter,population,gpt_available
0,AD-2020-Q1,0.0,AD,2020,Q1,79824,1
1,AD-2020-Q2,0.0,AD,2020,Q2,79824,1
2,AD-2020-Q3,0.0,AD,2020,Q3,79824,1
3,AD-2020-Q4,0.0,AD,2020,Q4,79824,1
4,AD-2021-Q1,0.0,AD,2021,Q1,79824,1
...,...,...,...,...,...,...,...
2555,ZW-2022-Q4,1641.0,ZW,2022,Q4,13061239,0
2556,ZW-2023-Q1,1890.0,ZW,2023,Q1,13061239,0
2557,ZW-2023-Q2,1897.0,ZW,2023,Q2,13061239,0
2558,ZW-2023-Q3,2036.0,ZW,2023,Q3,13061239,0


In [38]:
quarter_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4} # AGREGADO
grouped_df['quarter'] = grouped_df['quarter'].map(quarter_map).astype(int) # AGREGADO
grouped_df['year'] = grouped_df['year'].astype(int) # AGREGADO
grouped_df['year_quarter'] = grouped_df['year'].astype(str) + '-Q' + grouped_df['quarter'].astype(str)

In [39]:
# Function to convert quarter format to integer
def quarter_to_int(quarter_string):
    year, q = quarter_string.split('-')
    year = int(year)
    quarter_number = int(q[1])  # Q1, Q2, Q3, Q4 -> 1, 2, 3, 4
    base_year = 2020  # Adjust based on your balanced_df, or set dynamically
    return 4 * (year - base_year) + quarter_number

# Applying the function
grouped_df['quarter_int'] = grouped_df['year_quarter'].apply(quarter_to_int)
grouped_df

,unique_id,num_pushers,iso2_code,year,quarter,population,gpt_available,year_quarter,quarter_int
0,AD-2020-Q1,0.0,AD,2020,1,79824,1,2020-Q1,1
1,AD-2020-Q2,0.0,AD,2020,2,79824,1,2020-Q2,2
2,AD-2020-Q3,0.0,AD,2020,3,79824,1,2020-Q3,3
3,AD-2020-Q4,0.0,AD,2020,4,79824,1,2020-Q4,4
4,AD-2021-Q1,0.0,AD,2021,1,79824,1,2021-Q1,5
...,...,...,...,...,...,...,...,...,...
2555,ZW-2022-Q4,1641.0,ZW,2022,4,13061239,0,2022-Q4,12
2556,ZW-2023-Q1,1890.0,ZW,2023,1,13061239,0,2023-Q1,13
2557,ZW-2023-Q2,1897.0,ZW,2023,2,13061239,0,2023-Q2,14
2558,ZW-2023-Q3,2036.0,ZW,2023,3,13061239,0,2023-Q3,15


In [40]:
grouped_df.year = grouped_df.year.astype(int)

In [41]:
# Create pushes_pc
grouped_df["num_pushers_pc"] = (grouped_df["num_pushers"] / grouped_df["population"])*100000

# Aplicar la condición y crear la columna post_CI
grouped_df['post_CI'] = ((grouped_df['quarter_int'] >= 12) & (grouped_df['gpt_available'] == 1)).astype(int)


In [42]:
grouped_df['gpt_available_post1'] = grouped_df.gpt_available * grouped_df.post_CI
grouped_df

,unique_id,num_pushers,iso2_code,year,quarter,population,gpt_available,year_quarter,quarter_int,num_pushers_pc,post_CI,gpt_available_post1
0,AD-2020-Q1,0.0,AD,2020,1,79824,1,2020-Q1,1,0.000000,0,0
1,AD-2020-Q2,0.0,AD,2020,2,79824,1,2020-Q2,2,0.000000,0,0
2,AD-2020-Q3,0.0,AD,2020,3,79824,1,2020-Q3,3,0.000000,0,0
3,AD-2020-Q4,0.0,AD,2020,4,79824,1,2020-Q4,4,0.000000,0,0
4,AD-2021-Q1,0.0,AD,2021,1,79824,1,2021-Q1,5,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2555,ZW-2022-Q4,1641.0,ZW,2022,4,13061239,0,2022-Q4,12,12.563892,0,0
2556,ZW-2023-Q1,1890.0,ZW,2023,1,13061239,0,2023-Q1,13,14.470296,0,0
2557,ZW-2023-Q2,1897.0,ZW,2023,2,13061239,0,2023-Q2,14,14.523890,0,0
2558,ZW-2023-Q3,2036.0,ZW,2023,3,13061239,0,2023-Q3,15,15.588108,0,0


In [43]:
# Guardar dataframe
grouped_df.to_csv('../output/csv/Prueba.csv', index=False)